In [2]:
import csv
from pprint import pprint
from time import sleep

from selenium.webdriver import Chrome
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException, InvalidArgumentException

In [3]:
driver = Chrome("../chromedriver/chromedriver.exe")

In [3]:
driver.get("https://www.tokopedia.com/gigabyteofficial/gigabyte-motherboard-b550-aorus-pro-ac?whid=0")
driver.implicitly_wait(30)

In [4]:
searchbar = driver.find_element_by_xpath('//*[@id="search-container"]/form/div/div/div/input')
searchbar.click()
searchbar.send_keys("tv sony")
searchbar.send_keys(Keys.ENTER)
print("search for {}..".format("tv sony"))

search for tv sony..


In [5]:
def scroll_to_bottom():
    for i in range(7):
        driver.find_element_by_xpath("/html/body").send_keys(Keys.PAGE_DOWN)
        sleep(0.7)

In [6]:
try:
    overlay = driver.find_element_by_xpath("/html/body/div[5]/div[3]")
    /html/body/div[5]/div[6]
    overlay.click()
    scroll_to_bottom()
    driver.find_element_by_xpath("//*[contains(text(),'Oke, Lanjut Belanja')]").click()
except (NoSuchElementException, TimeoutException):
    pass

In [7]:
urls = driver.find_elements_by_xpath('//*[@id="zeus-root"]/div/div[2]/div/div[2]/div[4]/div/div/div/div/div/div/div/div[2]/a')
urls = [url.get_attribute('href') for url in urls]
promo_url = []

for url in urls[:]:
    if 'ta.tokopedia' in url:
        promo_url.append(url)
        urls.remove(url)
        
        

In [8]:
driver.get(urls[0])

In [14]:
separator = 'https%3A%2F%2F'
for url in promo_url:
    url = url.split(separator)
    url = url[1].replace("%2F", "/")
    urls.append(url)

len(urls)

80

In [19]:
title = driver.find_element_by_xpath('//*[@id="pdp_comp-product_content"]/div/h1').text
sold = driver.find_element_by_xpath('//*[@id="pdp_comp-product_content"]/div/div[1]/div/div[1]').text
price = driver.find_element_by_xpath('//*[@id="pdp_comp-product_content"]/div/div[2]/div').text
city = driver.find_element_by_xpath('//*[@id="pdp_comp-shipment"]/div/div/div[1]/div/b').text

result = {
    'title': title,
    'price': price,
    'sold': sold,
    'city': city
}
print(result)

{'title': 'SONY LED TV KD 85X8000H - SMART TV LED 85 INCH ANDROID TV 4K 85X8000', 'price': 'Rp41.999.000', 'sold': 'Terjual 34', 'city': 'Kota Tangerang Selatan'}
